In [31]:
from func_aux import *

In [32]:
def calculate_score_difference(estado, jogador):
    player_score = estado.state[estado.SOUTH_SCORE_PIT] if jogador == 0 else estado.state[estado.NORTH_SCORE_PIT]
    opponent_score = estado.state[estado.NORTH_SCORE_PIT] if jogador == 0 else estado.state[estado.SOUTH_SCORE_PIT]
    return player_score - opponent_score

def calculate_seeds_difference(estado, jogador):
    player_pits = estado.state[:estado.SOUTH_SCORE_PIT] if jogador == 0 else estado.state[estado.SOUTH_SCORE_PIT:estado.NORTH_SCORE_PIT]
    opponent_pits = estado.state[estado.SOUTH_SCORE_PIT:estado.NORTH_SCORE_PIT] if jogador == 0 else estado.state[:estado.SOUTH_SCORE_PIT]
    return sum(player_pits) - sum(opponent_pits)

def calculate_progress(estado, jogador):
    player_score = estado.state[estado.SOUTH_SCORE_PIT] if jogador == 0 else estado.state[estado.NORTH_SCORE_PIT]
    opponent_score = estado.state[estado.NORTH_SCORE_PIT] if jogador == 0 else estado.state[estado.SOUTH_SCORE_PIT]
    total_seeds = sum(estado.state[:estado.NORTH_SCORE_PIT])
    return 2 * (player_score - opponent_score) / total_seeds

def calculate_pits_difference(estado, jogador):
    player_pits = estado.state[:estado.SOUTH_SCORE_PIT] if jogador == 0 else estado.state[estado.SOUTH_SCORE_PIT:estado.NORTH_SCORE_PIT]
    opponent_pits = estado.state[estado.SOUTH_SCORE_PIT:estado.NORTH_SCORE_PIT] if jogador == 0 else estado.state[:estado.SOUTH_SCORE_PIT]
    
    player_pit_values = [min(pit, 6) for pit in player_pits]
    opponent_pit_values = [min(pit, 6) for pit in opponent_pits]
    
    return sum(opponent_pit_values) - sum(player_pit_values)

def calculate_opponent_capture_penalty(estado, jogador):
    next_states = [estado.real_move(move) for move in estado.get_legal_moves()]
    opponent_capture_penalty = 0
    
    for state in next_states:
        south_pieces = sum(state.state[:state.SOUTH_SCORE_PIT])
        north_pieces = sum(state.state[state.SOUTH_SCORE_PIT:state.NORTH_SCORE_PIT])
        opponent_capture_penalty += (south_pieces - north_pieces) * 0.05
    
    return opponent_capture_penalty

def func_44_otimo(estado, jogador):
    # Define weights for different evaluation factors
    SCORE_WEIGHT = 4
    SEEDS_WEIGHT = 1
    PROGRESS_WEIGHT = 2
    PITS_WEIGHT = 3

    # Calculate the final evaluation value
    evaluation = (
        SCORE_WEIGHT * calculate_score_difference(estado, jogador) +
        SEEDS_WEIGHT * calculate_seeds_difference(estado, jogador) +
        PROGRESS_WEIGHT * calculate_progress(estado, jogador) +
        PITS_WEIGHT * calculate_pits_difference(estado, jogador) -
        calculate_opponent_capture_penalty(estado, jogador)
    )

    return evaluation

In [33]:
def calculate_score_difference(estado, jogador):
    player_score = estado.state[estado.SOUTH_SCORE_PIT] if jogador == 0 else estado.state[estado.NORTH_SCORE_PIT]
    opponent_score = estado.state[estado.NORTH_SCORE_PIT] if jogador == 0 else estado.state[estado.SOUTH_SCORE_PIT]
    return player_score - opponent_score

def calculate_seeds_difference(estado, jogador):
    player_pits = estado.state[:estado.SOUTH_SCORE_PIT] if jogador == 0 else estado.state[estado.SOUTH_SCORE_PIT:estado.NORTH_SCORE_PIT]
    opponent_pits = estado.state[estado.SOUTH_SCORE_PIT:estado.NORTH_SCORE_PIT] if jogador == 0 else estado.state[:estado.SOUTH_SCORE_PIT]
    return sum(player_pits) - sum(opponent_pits)

def calculate_progress(estado, jogador):
    player_score = estado.state[estado.SOUTH_SCORE_PIT] if jogador == 0 else estado.state[estado.NORTH_SCORE_PIT]
    opponent_score = estado.state[estado.NORTH_SCORE_PIT] if jogador == 0 else estado.state[estado.SOUTH_SCORE_PIT]
    total_seeds = sum(estado.state[:estado.NORTH_SCORE_PIT])
    return 2 * (player_score - opponent_score) / total_seeds

def calculate_pits_difference(estado, jogador):
    player_pits = estado.state[:estado.SOUTH_SCORE_PIT] if jogador == 0 else estado.state[estado.SOUTH_SCORE_PIT:estado.NORTH_SCORE_PIT]
    opponent_pits = estado.state[estado.SOUTH_SCORE_PIT:estado.NORTH_SCORE_PIT] if jogador == 0 else estado.state[:estado.SOUTH_SCORE_PIT]
    
    player_pit_values = [min(pit, 6) for pit in player_pits]
    opponent_pit_values = [min(pit, 6) for pit in opponent_pits]
    
    return sum(opponent_pit_values) - sum(player_pit_values)

def calculate_opponent_capture_penalty(estado, jogador):
    next_states = [estado.real_move(move) for move in estado.get_legal_moves()]
    opponent_capture_penalty = 0
    
    for state in next_states:
        south_pieces = sum(state.state[:state.SOUTH_SCORE_PIT])
        north_pieces = sum(state.state[state.SOUTH_SCORE_PIT:state.NORTH_SCORE_PIT])
        opponent_capture_penalty += (south_pieces - north_pieces) * 0.05
    
    return opponent_capture_penalty

def func_44_default(estado, jogador):
    # Define weights for different evaluation factors
    SCORE_WEIGHT = 1
    SEEDS_WEIGHT = 1
    PROGRESS_WEIGHT = 1
    PITS_WEIGHT = 1

    # Calculate the final evaluation value
    evaluation = (
        SCORE_WEIGHT * calculate_score_difference(estado, jogador) +
        SEEDS_WEIGHT * calculate_seeds_difference(estado, jogador) +
        PROGRESS_WEIGHT * calculate_progress(estado, jogador) +
        PITS_WEIGHT * calculate_pits_difference(estado, jogador) -
        calculate_opponent_capture_penalty(estado, jogador)
    )

    return evaluation

In [34]:
def calculate_score_difference(estado, jogador):
    player_score = estado.state[estado.SOUTH_SCORE_PIT] if jogador == 0 else estado.state[estado.NORTH_SCORE_PIT]
    opponent_score = estado.state[estado.NORTH_SCORE_PIT] if jogador == 0 else estado.state[estado.SOUTH_SCORE_PIT]
    return player_score - opponent_score

def calculate_seeds_difference(estado, jogador):
    player_pits = estado.state[:estado.SOUTH_SCORE_PIT] if jogador == 0 else estado.state[estado.SOUTH_SCORE_PIT:estado.NORTH_SCORE_PIT]
    opponent_pits = estado.state[estado.SOUTH_SCORE_PIT:estado.NORTH_SCORE_PIT] if jogador == 0 else estado.state[:estado.SOUTH_SCORE_PIT]
    return sum(player_pits) - sum(opponent_pits)

def calculate_progress(estado, jogador):
    player_score = estado.state[estado.SOUTH_SCORE_PIT] if jogador == 0 else estado.state[estado.NORTH_SCORE_PIT]
    opponent_score = estado.state[estado.NORTH_SCORE_PIT] if jogador == 0 else estado.state[estado.SOUTH_SCORE_PIT]
    total_seeds = sum(estado.state[:estado.NORTH_SCORE_PIT])
    return 2 * (player_score - opponent_score) / total_seeds

def calculate_pits_difference(estado, jogador):
    player_pits = estado.state[:estado.SOUTH_SCORE_PIT] if jogador == 0 else estado.state[estado.SOUTH_SCORE_PIT:estado.NORTH_SCORE_PIT]
    opponent_pits = estado.state[estado.SOUTH_SCORE_PIT:estado.NORTH_SCORE_PIT] if jogador == 0 else estado.state[:estado.SOUTH_SCORE_PIT]
    
    player_pit_values = [min(pit, 6) for pit in player_pits]
    opponent_pit_values = [min(pit, 6) for pit in opponent_pits]
    
    return sum(opponent_pit_values) - sum(player_pit_values)

def calculate_opponent_capture_penalty(estado, jogador):
    next_states = [estado.real_move(move) for move in estado.get_legal_moves()]
    opponent_capture_penalty = 0
    
    for state in next_states:
        south_pieces = sum(state.state[:state.SOUTH_SCORE_PIT])
        north_pieces = sum(state.state[state.SOUTH_SCORE_PIT:state.NORTH_SCORE_PIT])
        opponent_capture_penalty += (south_pieces - north_pieces) * 0.05
    
    return opponent_capture_penalty

def func_44_teste(estado, jogador):
    # Define weights for different evaluation factors
    SCORE_WEIGHT = 4
    SEEDS_WEIGHT = 5
    PROGRESS_WEIGHT = 3
    PITS_WEIGHT = 5

    # Calculate the final evaluation value
    evaluation = (
        SCORE_WEIGHT * calculate_score_difference(estado, jogador) +
        SEEDS_WEIGHT * calculate_seeds_difference(estado, jogador) +
        PROGRESS_WEIGHT * calculate_progress(estado, jogador) +
        PITS_WEIGHT * calculate_pits_difference(estado, jogador) -
        calculate_opponent_capture_penalty(estado, jogador)
    )

    return evaluation



#  Testes entre Bonecos

In [ ]:
default = JogadorAlfaBeta('Defalut', 6, func_44_default)
otimo = JogadorAlfaBeta('Otimo', 6, func_44_otimo)
teste = JogadorAlfaBeta('Teste', 6, func_44_teste)

jogo=Kalah(10)
print(jogaNpares(jogo,100,default,otimo))
print("")
print("")
print(jogaNpares(jogo,100,default,teste))
print("")
print("")
print(jogaNpares(jogo,100,otimo,teste))

({'Defalut': 18, 'Otimo': 81, 'Empate': 1}, {'Defalut': 20, 'Otimo': 74, 'Empate': 6}, {'Defalut': 38, 'Otimo': 155, 'Empate': 7}, {'Defalut': 121, 'Otimo': 472})


